<a href="https://colab.research.google.com/github/hnishi/jupyterbook-hnishi/blob/update_bert_fine_tuning/jupyterbook_hnishi/language-models/fine_tune_jp_bert_part01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# huggingface transformers を使って日本語 BERT モデルをファインチューニングして感情分析 (with google colab) part01

本記事では、日本語 BERT モデルをファインチューニングして感情分析する方法を解説します。

BERT の詳細な解説は、この記事のスコープ外とします。

この記事は、part01 です。

[part02](https://jupyterbook.hnishi.com/language-models/fine_tune_jp_bert_part02.html) では、まとまったデータセットを使って実際に学習と評価を行っています。

## 参考

- [huggingface transformers ドキュメント](https://huggingface.co/transformers/)
- [BERT 論文](https://arxiv.org/abs/1810.04805)
- [Fine-tuning a BERT model with transformers](https://towardsdatascience.com/fine-tuning-a-bert-model-with-transformers-c8e49c4e008b)

## 必要なライブラリのインストール

In [1]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 25.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from transformers import AdamW

## 日本語 BERT の簡単なチュートリアル

最初に、huggingface transformers を使った日本語 BERT pre-trained model の使い方や fine tuning の方法を、簡単に見ていきます。

今回試す事前学習済みモデルとしては、 bert-large-japanese　を利用してみます。

---

**補足**

最近 (2021-03-05)、東北大学のグループから BERT の日本語 pre-trained models の [version 2](https://github.com/cl-tohoku/bert-japanese/releases/tag/v2.0) が公開されています。

このリリースでは、より大きなアーキテクチャの bert-large-japanese モデルも公開されています。

- cl-tohoku/bert-base-japanese-v2
- cl-tohoku/bert-large-japanese

```
BERT-base models consist of 12 layers, 768 dimensions of hidden states, and 12 attention heads.
BERT-large models consist of 24 layers, 1024 dimensions of hidden states, and 16 attention heads.
```

- https://huggingface.co/cl-tohoku
- https://github.com/cl-tohoku/bert-japanese

### Pre-trained Model を使って推論

BERT なので、mask された token を予測するように学習されています。

したがって、pre-trained model を使って、文章中の穴埋め (文章中の欠損箇所の予測) を行うことができます。

以下の２種類のモデルを使って推論してみました。

- cl-tohoku/bert-large-japanese
- bert-base-multilingual-uncased (BERT の多言語モデル)

結果、 `bert-base-multilingual-uncased` のほうが自然な文章となりました。

この違いは、学習に使用されたデータセットに依存しているのかと思いましたが、どちらのモデルも wikipedia をデータセットとして利用しているので、謎です。

In [3]:
model_name = "cl-tohoku/bert-large-japanese"

tokenizer = BertTokenizer.from_pretrained(model_name)

unmasker = pipeline('fill-mask', model=model_name, tokenizer=tokenizer)
unmasker("こんにちは、私は[MASK]モデルです。")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


Some weights of the model checkpoint at cl-tohoku/bert-large-japanese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.13797813653945923,
  'token': 860,
  'token_str': 'お',
  'sequence': 'こんにちは 、 私 は お モデルです 。'},
 {'score': 0.09143563359975815,
  'token': 856,
  'token_str': 'う',
  'sequence': 'こんにちは 、 私 は う モデルです 。'},
 {'score': 0.036211829632520676,
  'token': 11668,
  'token_str': 'こう',
  'sequence': 'こんにちは 、 私 は こう モデルです 。'},
 {'score': 0.028521692380309105,
  'token': 828,
  'token_str': '、',
  'sequence': 'こんにちは 、 私 は 、 モデルです 。'},
 {'score': 0.026474272832274437,
  'token': 24523,
  'token_str': 'いら',
  'sequence': 'こんにちは 、 私 は いら モデルです 。'}]

In [4]:
model_name = "bert-base-multilingual-uncased"

unmasker = pipeline('fill-mask', model=model_name)
unmasker("こんにちは、私は[MASK]モデルです。")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.21041347086429596,
  'token': 1482,
  'token_str': '、',
  'sequence': 'こんにちは 、 私 は 、 モテルてす 。'},
 {'score': 0.12128561735153198,
  'token': 2051,
  'token_str': '元',
  'sequence': 'こんにちは 、 私 は 元 モテルてす 。'},
 {'score': 0.03390878438949585,
  'token': 2178,
  'token_str': '初',
  'sequence': 'こんにちは 、 私 は 初 モテルてす 。'},
 {'score': 0.029899438843131065,
  'token': 3014,
  'token_str': '女',
  'sequence': 'こんにちは 、 私 は 女 モテルてす 。'},
 {'score': 0.021887270733714104,
  'token': 5846,
  'token_str': '男',
  'sequence': 'こんにちは 、 私 は 男 モテルてす 。'}]

## テキスト分類のための Fine Tuning の手順


以下、簡易的に 3 種類のラベル (positive: 2, neutral: 1, negative: 0) のデータを使って fine tuning を行います。

In [5]:
# 確認用のデータセット
df = pd.DataFrame([{"text": "私はこの映画をみることができて、とても嬉しい。", "label": 2},
                              {"text": "今日の晩御飯は何だろう。", "label": 1},
                              {"text": "猫に足を噛まれて痛い。", "label": 0}])
df

,text,label
0,私はこの映画をみることができて、とても嬉しい。,2
1,今日の晩御飯は何だろう。,1
2,猫に足を噛まれて痛い。,0


In [6]:
train_docs = df["text"].tolist()
train_labels = df["label"].tolist()

## 学習

同時にダウンロードされるトークナイザーを利用して、データセットの text の encoding を行います。

In [7]:
# Ref: https://huggingface.co/transformers/training.html#pytorch

model_name = "cl-tohoku/bert-large-japanese"

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at cl-tohoku/bert-large-japanese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

In [8]:
encodings = tokenizer(train_docs, return_tensors='pt', padding=True, truncation=True, max_length=128)
input_ids = encodings['input_ids']
attention_mask = encodings['attention_mask']

In [9]:
# Fine-tuning in native PyTorch

# > the AdamW() optimizer which implements gradient bias correction as well as weight decay.
optimizer = AdamW(model.parameters(), lr=1e-5)

labels = torch.tensor(train_labels).unsqueeze(0)
outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
loss = outputs.loss
loss.backward()
optimizer.step()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Fine Tune したモデルで推論

学習に使ったデータを入力して推論してみます。

In [10]:
sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [11]:
sentiment_analyzer("これは、テストのための文章です")

[{'label': 'LABEL_0', 'score': 0.3716263175010681}]

In [12]:
_ = list(map(lambda x: print(f"{x}: {sentiment_analyzer(x)}"), train_docs))

私はこの映画をみることができて、とても嬉しい。: [{'label': 'LABEL_2', 'score': 0.4420872926712036}]
今日の晩御飯は何だろう。: [{'label': 'LABEL_1', 'score': 0.6111928224563599}]
猫に足を噛まれて痛い。: [{'label': 'LABEL_0', 'score': 0.48623162508010864}]


スコアは低いですが、学習に使ったデータセットに関して、正しいラベルが予測できていることを確認できました。

## まとめ

簡単な文章とラベルを用意して fine tuning する方法を記載しました。

[次の記事](https://jupyterbook.hnishi.com/language-models/fine_tune_jp_bert_part02.html) では、より大きなデータセットを使って、より時間のかかる学習を試してみたいと思います。